<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/5_11%ED%8C%8C%EC%9D%B4%EC%8D%AC(CNN%2CImageDataGenerator%2Cimg_to_array).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
5.11

overfitting
- 학습 데이터에 대해 과하게 학습된 상황
- 학습 데이터에 대해서는 정확도가 좋은데 학습데이터 이외의 데이터에 대해서는 모델이 잘 작동하지 않는다.
- 학습데이터가 부족하거나 데이터의 특성에 비해 모델이 너무 복잡한 경우 발생한다

underfitting
- 학습데이터도 학습을 하지 못한 상태를 의미
- 학습 반복횟수가 너무 작을 때 발생할 수 있다.
- 데이터의 특성에 비해 모델이 너무 간단하게 설계되어 있는 경우
- 데이터 양이 너무 작응 경우

데이터 증강(Data Augmentation)
- CNN 모델의 성능을 높이고 overfitting을 줄일 수 있는 좋은 방법은 다양한 유형의 학습 이미지 데이터의 양을 늘리는 것
- data augmentation은 학습시에 원본 이미지에 다양한 변형을 가해서 학습 이미지 데이터를 늘리는 것과 유사한 효과를 발휘하고자 하는 기법
- data augmentation은 원본 학습 이미지 개수를 늘리는 것이 아니고 학습시 마다 개별 원본 이미지를 변형해서 학습을 수행한다.

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

ImageDataGenerator(
    rotation_range=10, # 이미지 회전(0~180)
    width_shift_range=0.2, # 이미지를 수평(좌우) 방향으로 랜덤하게 평행 이동
    height_shift_range=0.2, # 이미지를 수직(상하) 방향으로 랜덤하게 평행 이동
    shear_range=0.1, # y축 방향으로 각도를 증가시켜 이미지 변형
    zoom_range=0.1, # 확대, 축소, 1보다 작은 경우 확장, 1보다 큰 값은 축소
    horizontal_flip=True, # True로 설정하는 경우 50% 학률로 이미지 수평(좌우)으로 뒤집는다.
    #vertical_flip=True 상하반전
    fill_mode = 'nearest', # 이미지를 회전, 이동, 축소 할 때 생기는 공간을 채우는 방식
                #'nearest' : 가장 빈공간에 가장 근접한 픽셀로 채우기
                #'reflect' : 빈 공간 만큼 영역을 근처 공간으로 채우되 거울로 반사되는 이미지 처럼 채운다.
                #'wrap': 빈 공간을 잘려나간 이미지로 채움
                # 'constant' : 특정 필셀값으로 채운다. cval=0 검은색으로 채운다
    brightness_range=(0.1,0.9) # 밝기 조절 0 ~ 1 사이의 값, 0에 가까울수록 어둡고 1에 가까울수록 밝다
    channel_shift_range = 100 # R,G,B값을 -100~100 사이의 임의의 값을 더하여 변환
    )

datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.2,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
img = load_img('c:/img/cat_1.jpg')
x = img_to_array(img)
x.shape
# (194,259,3) -> (1,194,259,3) 으로 변환작업 필요
x = x[np.newaxis,:,:,:] # 변환
x.shape

i = 0
for batch in datagen.flow(x,save_to_dir='c:/data/',save_prefix='20220511',save_format='jpg'): # 이미지를 변형시키면서 파일로 저장
    i += 1 
    if i > 3:
        break

# 개, 고양이 분류
import os
os.listdir('c:/data/cats_dogs') # 디렉토리 확인
os.listdir('c:/data/cats_dogs/train')
os.listdir('c:/data/cats_dogs/train/cats') # 디렉토리 안에 파일 확인

1. binary
train_path = 'c:/data/cats_dogs/train'
validation_path = 'c:/data/cats_dogs/validation'

training_datagen = ImageDataGenerator( # 학습
    rescale = 1./255.,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255.) # 검증

training_generator = training_datagen.flow_from_directory(train_path,
                                     batch_size=32,
                                     target_size=(64,64),
                                     class_mode='binary')

training_generator.classes # 레이블 된 것을 보여줌
training_generator.class_indices # 어떻게 레이블이 적용됐는지 딕셔너리로 보여줌


validation_generator = validation_datagen.flow_from_directory(validation_path,
                                     batch_size=32, # training이랑 틀려도 됨
                                     shuffle=True, # 섞기
                                     target_size=(64,64),
                                     class_mode='binary')

validation_generator.classes # 레이블 된 것을 보여줌
validation_generator.class_indices # 어떻게 레이블이 적용됐는지 딕셔너리로 보여줌

#모델링
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, Dense, Dropout # Dropout은 오버피팅을 줄이는데 사용
import tensorflow as tf

model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(64,64,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(64,64,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2))) # 꼭 해야하는 것은 아님?
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(64,64,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5)) # 50% 정도 없애자
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(training_generator,validation_data=validation_generator,epochs=20) 

# 새로운 이미지로 테스트
img = load_img('c:/img/cat_1.jpg')
x = img_to_array(img)
x.shape
x = tf.image.resize(x,[64,64])
x.shape
(64,64,3) -> (1,64,64,3)
x = np.array([x])
x.shape

predict = model.predict(x.astype('float')/255) >= 0.5 # 예측된 값 0.5보다 크거나 같으면 True 개, 작으면 false 고양이 
list(training_generator.class_indices.keys())[int(predict)]

# 새로운 이미지로 테스트2
img = load_img('c:/img/dog_1.jpg')
x = img_to_array(img)
x.shape
x = tf.image.resize(x,[64,64])
x.shape
(64,64,3) -> (1,64,64,3)
x = np.array([x])
x.shape

predict = model.predict(x.astype('float')/255) >= 0.5 # 예측된 값 0.5보다 크거나 같으면 True 개, 작으면 false 고양이 
list(training_generator.class_indices.keys())[int(predict)]


2. categorical( 레이블이 여러개일 때 )
train_path = 'c:/data/cats_dogs/train'
validation_path = 'c:/data/cats_dogs/validation'

training_datagen = ImageDataGenerator( # 학습
    rescale = 1./255.,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255.) # 검증

training_generator = training_datagen.flow_from_directory(train_path,
                                     batch_size=32,
                                     target_size=(64,64),
                                     class_mode='categorical')

training_generator.classes # 레이블 된 것을 보여줌
training_generator.class_indices # 어떻게 레이블이 적용됐는지 딕셔너리로 보여줌


validation_generator = validation_datagen.flow_from_directory(validation_path,
                                     batch_size=32, # training이랑 틀려도 됨
                                     shuffle=True, # 섞기
                                     target_size=(64,64),
                                     class_mode='categorical')

validation_generator.classes # 레이블 된 것을 보여줌
validation_generator.class_indices # 어떻게 레이블이 적용됐는지 딕셔너리로 보여줌

#모델링
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, Dense, Dropout # Dropout은 오버피팅을 줄이는데 사용
import tensorflow as tf

model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(64,64,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(64,64,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2))) # 꼭 해야하는 것은 아님?
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(64,64,3),activation='relu')) 
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5)) # 50% 정도 없애자
model.add(Dense(32,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.summary()

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(training_generator,validation_data=validation_generator,epochs=20) 

# 새로운 이미지로 테스트
img = load_img('c:/img/cat_1.jpg')
x = img_to_array(img)
x.shape
x = tf.image.resize(x,[64,64])
x.shape
(64,64,3) -> (1,64,64,3)
x = np.array([x])
x.shape

predict = model.predict(x.astype('float')/255)
list(training_generator.class_indices.keys())[np.argmax(predict)]


img = load_img('c:/img/dog_1.jpg')
x = img_to_array(img)
x.shape
x = tf.image.resize(x,[64,64])
x.shape
(64,64,3) -> (1,64,64,3)
x = np.array([x])
x.shape

predict = model.predict(x.astype('float')/255)
list(training_generator.class_indices.keys())[np.argmax(predict)]

 
# 2개만 분류할 때 binary를 사용하면 Dense에서 1, sigmoid 사용
# 여러개 분류할 때 categorical을 사용하면 2(2개일때), softmax 사용